In [1]:
!kallisto version

kallisto, version 0.46.2


# Pseudoalign Samples

This notebook will pseudoalign the samples and map them to a list of genes and list of counts for each gene. The final product will be a dataframe, where the genes are rows, and the columns represents samples 1-30. However, we will test this process with just 1 sample first.

## Step 1: Indexing

We need to create an index using kallisto. Kallisto reads the sequences from the specified .fna file, then builds an index based on these sequences. The index is a data structure that Kallisto uses later for quickly aligning reads from RNA-Seq data to the transcriptome. The created index is saved as 'output_indexed.idx'. Once this index is built, it can be used for quantifying transcript abundances from the RNA-Seq data using 'kallisto quant', which is the next step.

In [2]:
#Create index using kallisto
#Took 8 1/2 minutes to run
!kallisto index -i "/mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/output_indexed.idx" "/mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/rna.fna"



[build] loading fasta file /mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/rna.fna
[build] k-mer length: 31
^C


## Step 2: Quantifying Transcript Abundances

The next step is to quantify transcript abundances. We read in the paired-end RNA-Seq FASTQ files for the first sample, which are in FASTQ format that have been compressed with gzip. Kallisto uses the index file specified by '-i' to align the RNA-Seq reads from the FASTQ files. Then it quantifies the abundance of transcripts based on this alignment. The results include estimated counts and other statistics, which are saved in the output directory specified by '-o'. The data is from paired-end sequencing, so kallisto will treat the 2 FASTQ files as paired reads, aligning them in a manner that considers their paired nature. The output is a file called 'abundance.tsv' which contains the estimated transcript abundances, among other data.

In [11]:
#Took 93 minutes to run
!kallisto quant -i "/mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/output_indexed.idx" -o "/mnt/d/kallisto" "/mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R1_001.fastq.gz" "/mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R2_001.fastq.gz"



Error: kallisto index file not found /mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/output_indexed.idx
Error: file not found /mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R1_001.fastq.gz
Error: file not found /mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R2_001.fastq.gz
Error: could not create directory /mnt/d/kallisto

Usage: kallisto quant [arguments] FASTQ-files

Required arguments:
-i, --index=STRING            Filename for the kallisto index to be used for
                              quantification
-o, --output-dir=STRING       Directory to write output to

Optional arguments:
    --bias                    Perform sequence based bias correction
-b, --bootstrap-samples=INT   Number of bootstrap samples (default: 0)
    --seed=INT                Seed for the bootstrap sampling (default: 42)
    --plaintext               Output plaintext instead of HDF5
    --fusion                  Search for fusions for Pizzly
    --single                  Quantify single

## Step 3: Import Data into a Pandas DataFrame

We take the 'abundance.tsv' file and and load it into a dataframe so we can inspect the data.

In [3]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
sample_1_df = pd.read_csv("/mnt/d/kallisto/abundance.tsv", sep='\t')
sample_1_df.head()

/tmp/ipykernel_163/2196393233.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/d/kallisto/abundance.tsv'

## Step 4: Data Manipulation

Next, we can select the columns 'target_id' and 'est_counts', which are the raw estimated counts. Then we can set 'target_id' as the index of 'gene_counts' dataframe. We do some renaming of the headers, and we get the desired output for 1 sample. Essentially, we are extracting the gene/transcript IDs and their corresponding expression measures from a larger dataset, setting up a concise dataframe for further analysis.

In [14]:
gene_counts = sample_1_df[['target_id', 'est_counts']]
gene_counts.set_index('target_id', inplace=True)
gene_counts.rename(columns={'est_counts': 'Sample_1'}, inplace=True)


/tmp/ipykernel_10897/929896097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_counts.rename(columns={'tpm': 'Sample_1'}, inplace=True)  # or 'est_counts' as per your data


In [16]:
gene_counts

,Sample_1
target_id,
NM_000014.6,0.000000
NM_000015.3,0.276341
NM_000016.6,3.067990
NM_000017.4,9.473520
NM_000018.4,90.751800
...,...
XR_953251.3,0.000000
XR_953252.3,0.000000
XR_953253.3,0.098578


# Psuedoalignment for all samples

We can do the above process for all of the samples now.

In [2]:
# This creates a list from 1 to 19 and adds 28, 29, 30
samples = list(range(1, 20)) + [28, 29, 30] 

In [3]:
import os

samples = list(range(1, 20)) + [28, 29, 30]  # Sample numbers

for sample in samples:
    sample_str = f"{sample:04d}"  # Formats the sample number as four digits

    r1_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R1_001.fastq.gz"
    r2_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R2_001.fastq.gz"

    output_dir = f"/mnt/d/kallisto/sample_{sample_str}"

    if os.path.exists(r1_path) and os.path.exists(r2_path):
        # Run kallisto quant for each sample
        !kallisto quant -i "/mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/output_indexed.idx" -o "{output_dir}" "{r1_path}" "{r2_path}"
    else:
        print(f"Files for Sample {sample_str} not found, skipping...")




[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 185,121
[index] number of k-mers: 142,403,527
[index] number of equivalence classes: 657,654
[quant] running in paired-end mode
[quant] will process pair 1: /mnt/e/20231030 - 10051-NM/10051-NM-0001_S1_L005_R1_001.fastq.gz
                             /mnt/e/20231030 - 10051-NM/10051-NM-0001_S1_L005_R2_001.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 207,412,414 reads, 107,409,028 reads pseudoaligned
[quant] estimated average fragment length: 188.166
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 1,484 rounds


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 185,121
[index] number of k-mers: 142,403,527
[index] number of equivalence classes: 657,654
[quant] running in paired-end mode
[quant] will process pa

Trying to process remaining samples as it crashed after finishing sample 14.

In [1]:
import os

samples = list(range(1, 20)) + [28, 29, 30]  # Sample numbers

for sample in samples:
    sample_str = f"{sample:04d}"  # Formats the sample number as four digits
    output_dir = f"/mnt/d/kallisto/sample_{sample_str}"
    abundance_file = f"{output_dir}/abundance.tsv"

    if not os.path.exists(abundance_file):  # Check if the abundance file already exists
        r1_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R1_001.fastq.gz"
        r2_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R2_001.fastq.gz"

        if os.path.exists(r1_path) and os.path.exists(r2_path):
            # Run kallisto quant for each sample
            !kallisto quant -i "/mnt/d/fastq/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/output_indexed.idx" -o "{output_dir}" "{r1_path}" "{r2_path}"
        else:
            print(f"FASTQ files for Sample {sample_str} not found, skipping...")
    else:
        print(f"Output already exists for Sample {sample_str}, skipping...")

FASTQ files for Sample 0001 not found, skipping...
FASTQ files for Sample 0002 not found, skipping...
FASTQ files for Sample 0003 not found, skipping...
FASTQ files for Sample 0004 not found, skipping...
FASTQ files for Sample 0005 not found, skipping...
FASTQ files for Sample 0006 not found, skipping...
FASTQ files for Sample 0007 not found, skipping...
FASTQ files for Sample 0008 not found, skipping...
FASTQ files for Sample 0009 not found, skipping...
FASTQ files for Sample 0010 not found, skipping...
FASTQ files for Sample 0011 not found, skipping...
FASTQ files for Sample 0012 not found, skipping...
FASTQ files for Sample 0013 not found, skipping...
FASTQ files for Sample 0014 not found, skipping...
FASTQ files for Sample 0015 not found, skipping...
FASTQ files for Sample 0016 not found, skipping...
FASTQ files for Sample 0017 not found, skipping...
FASTQ files for Sample 0018 not found, skipping...
FASTQ files for Sample 0019 not found, skipping...
FASTQ files for Sample 0028 not

In [7]:
import pandas as pd

samples = list(range(1, 20)) + [28, 29, 30]  # Sample numbers
combined_df = pd.DataFrame()

for sample in samples:
    sample_str = f"{sample:04d}"
    abundance_path = f"/mnt/d/kallisto/sample_{sample_str}/abundance.tsv"
    
    if os.path.exists(abundance_path):
        sample_df = pd.read_csv(abundance_path, sep='\t')
        sample_df = sample_df[['target_id', 'est_counts']].rename(columns={'est_counts': f'Sample_{sample_str}'})
        sample_df.set_index('target_id', inplace=True)

        if combined_df.empty:
            combined_df = sample_df
        else:
            combined_df = combined_df.join(sample_df, how='outer')
    else:
        print(f"Abundance file for Sample {sample_str} not found, skipping...")


In [8]:
combined_df

,Sample_0001,Sample_0002,Sample_0003,Sample_0004,Sample_0005,Sample_0006,Sample_0007,Sample_0008,Sample_0009,Sample_0010,...,Sample_0013,Sample_0014,Sample_0015,Sample_0016,Sample_0017,Sample_0018,Sample_0019,Sample_0028,Sample_0029,Sample_0030
target_id,,,,,,,,,,,,,,,,,,,,,
NM_000014.6,0.0000,0.0000,0.000,0.000,0.000000,0.0000,17.6779,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,106.4040,20.8688,11.689100,0.0000
NM_000015.3,53.0000,21.3756,204.000,40.000,65.000000,26.4979,40.0000,22.0000,86.0000,234.0000,...,143.0000,127.0000,133.2920,145.4330,139.0000,77.0000,101.0000,183.0000,144.251000,126.0000
NM_000016.6,1111.5300,633.1070,0.000,685.444,941.400000,700.2260,800.0770,1319.2000,877.3860,2593.1300,...,2226.5200,2023.3900,2138.0100,2178.3400,1815.4800,1600.4100,1311.3700,2062.5300,1095.380000,2197.4800
NM_000017.4,2766.9300,1395.6200,548.174,1393.820,2528.330000,1978.6700,1899.0600,1861.7100,1307.0000,2078.7900,...,1826.9200,2384.5200,2385.2700,2400.4300,2145.7500,1887.3400,3241.2300,1668.9900,1705.860000,1768.3700
NM_000018.4,31658.6000,17213.7000,4179.900,19550.400,29982.600000,22879.0000,24013.5000,22633.2000,30253.7000,35009.0000,...,29471.8000,31795.3000,32138.3000,26845.2000,24289.1000,23880.0000,31656.9000,27760.3000,24063.300000,21014.9000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XR_953251.3,0.0000,0.0000,0.000,0.000,0.066334,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,28.9198
XR_953252.3,0.0000,0.0000,0.000,0.000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.510645,0.0000
XR_953253.3,65.4392,42.2042,232.735,37.840,29.089400,24.0502,37.6286,38.1789,63.3103,23.9241,...,24.8345,33.6907,27.5614,38.1278,21.1381,40.0857,53.2241,40.7266,44.493600,54.1422


In [9]:
combined_df.to_csv('/mnt/d/kallisto/pseudoalign_df.csv')


# Redo Process with Different Index

The same process will be run with a different index now.

In [2]:
#Create index using kallisto
#Took 6 1/2 minutes to run
!kallisto index -i "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/output_index.idx" "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/Homo_sapiens.GRCh38.cdna.all.fa"


[build] loading fasta file /mnt/e/fastq/Homo_sapiens.GRCh38.cdna.all.fa/Homo_sapiens.GRCh38.cdna.all.fa
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 1525 target sequences
[build] warning: replaced 100005 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 1233435 contigs and contains 116726086 k-mers 



In [3]:
#Took 134 minutes to run
!kallisto quant -i "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/output_index.idx" -o "/mnt/d/kallisto_Homo_sapiens" "/mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R1_001.fastq.gz" "/mnt/d/20231030 - 10051-NM/10051-NM-0001_S1_L005_R2_001.fastq.gz"


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 207,249
[index] number of k-mers: 116,726,086
[index] number of equivalence classes: 848,752
[quant] running in paired-end mode
[quant] will process pair 1: /mnt/e/20231030 - 10051-NM/10051-NM-0001_S1_L005_R1_001.fastq.gz
                             /mnt/e/20231030 - 10051-NM/10051-NM-0001_S1_L005_R2_001.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 207,412,414 reads, 96,941,450 reads pseudoaligned
[quant] estimated average fragment length: 174.509
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 1,327 rounds



# This code includes sample 19

In [2]:
#Ran for 2000 minutes before having to stop.
import os

samples = list(range(1, 31))  # Sample numbers

for sample in samples:
    sample_str = f"{sample:04d}"  # Formats the sample number as four digits
    output_dir = f"/mnt/d/kallisto_Homo_sapiens/sample_{sample_str}"
    abundance_file = f"{output_dir}/abundance.tsv"

    if not os.path.exists(abundance_file):  # Check if the abundance file already exists
        r1_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R1_001.fastq.gz"
        r2_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R2_001.fastq.gz"

        if os.path.exists(r1_path) and os.path.exists(r2_path):
            # Run kallisto quant for each sample
            !kallisto quant -i "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/output_index.idx" -o "{output_dir}" "{r1_path}" "{r2_path}"
        else:
            print(f"FASTQ files for Sample {sample_str} not found, skipping...")
    else:
        print(f"Output already exists for Sample {sample_str}, skipping...")

Output already exists for Sample 0001, skipping...
Output already exists for Sample 0002, skipping...
Output already exists for Sample 0003, skipping...
Output already exists for Sample 0004, skipping...
Output already exists for Sample 0005, skipping...
Output already exists for Sample 0006, skipping...
Output already exists for Sample 0007, skipping...
Output already exists for Sample 0008, skipping...
Output already exists for Sample 0009, skipping...
Output already exists for Sample 0010, skipping...
Output already exists for Sample 0011, skipping...
Output already exists for Sample 0012, skipping...
Output already exists for Sample 0013, skipping...
Output already exists for Sample 0014, skipping...
Output already exists for Sample 0015, skipping...
Output already exists for Sample 0016, skipping...
Output already exists for Sample 0017, skipping...
Output already exists for Sample 0018, skipping...

[quant] fragment length distribution will be estimated from the data
[index] k-me

The quantification process is unable to run for sample 19 via Python and Jupter Notebook for undetermined reasons. So, the code below will run the algorithm for all samples but sample 19.

In [ ]:
#Ran for 2000 minutes before having to stop.
import os

samples = list(range(1, 19)) + [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]  # Sample numbers

for sample in samples:
    sample_str = f"{sample:04d}"  # Formats the sample number as four digits
    output_dir = f"/mnt/d/kallisto_Homo_sapiens/sample_{sample_str}"
    abundance_file = f"{output_dir}/abundance.tsv"

    if not os.path.exists(abundance_file):  # Check if the abundance file already exists
        r1_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R1_001.fastq.gz"
        r2_path = f"/mnt/d/20231030 - 10051-NM/10051-NM-{sample_str}_S1_L005_R2_001.fastq.gz"

        if os.path.exists(r1_path) and os.path.exists(r2_path):
            # Run kallisto quant for each sample
            !kallisto quant -i "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/output_index.idx" -o "{output_dir}" "{r1_path}" "{r2_path}"
        else:
            print(f"FASTQ files for Sample {sample_str} not found, skipping...")
    else:
        print(f"Output already exists for Sample {sample_str}, skipping...")

Sample 19 needs to run in the windows terminal. Below are the commands.

In [ ]:
kallisto quant -i "/mnt/d/fastq/Homo_sapiens.GRCh38.cdna.all.fa/output_index.idx" -o "/mnt/d/kallisto_Homo_sapiens/sample_0019" "/mnt/d/20231030 - 10051-NM/10051-NM-0019_S1_L005_R1_001.fastq.gz" "/mnt/d/20231030 - 10051-NM/10051-NM-0019_S1_L005_R2_001.fastq.gz"